#### Building CNN to Detect Facial Key Points

In [18]:
import torchvision
import torch.nn as nn
import torch
import torch.nn.functional as F
from torchvision import transforms, models, datasets
from torchsummary import summary
import numpy as np, pandas as pd, os, glob, cv2
from torch.utils.data import TensorDataset,DataLoader,Dataset
from copy import deepcopy
from mpl_toolkits.mplot3d import Axes3D

In [19]:
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import cluster
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [20]:
#!git clone https://github.com/udacity/P1_Facial_Keypoints.git

In [23]:
train_data = 'data/data/training/'

In [22]:
class FaceData(Dataset):
    def __init__(self,df):
        super(FaceData).__init__()
        self.df = df
        self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    def __len__(self):
        return len(self.df)
    def __getitem__(self,indx):
        img_path = 'data/data/training/' + self.df.iloc[ix,0]
        img = cv2.imread(img_path)/255
    